In [1]:
import tensorflow as tf
import numpy as np
import os
from sklearn.metrics import roc_auc_score, recall_score
from tqdm import tqdm

In [18]:

model_path = 'models/final/Large_model.keras'
converted_model_path = 'models/final/Large_model.tflite'

In [44]:
model = tf.keras.models.load_model(model_path)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter.experimental_new_converter = True
converter.experimental_lower_tensor_list_ops = False
tflite_quant_model = converter.convert()
with open(converted_model_path, "wb") as f:
    f.write(tflite_quant_model)

INFO:tensorflow:Assets written to: C:\Users\stime\AppData\Local\Temp\tmphf3hhn_o\assets


INFO:tensorflow:Assets written to: C:\Users\stime\AppData\Local\Temp\tmphf3hhn_o\assets


In [45]:
print("Float model in Mb:", os.path.getsize(model_path) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(converted_model_path) / float(2**20))
print("Compression ratio:", os.path.getsize(model_path)/os.path.getsize(converted_model_path))

Float model in Mb: 11.195030212402344
Quantized model in Mb: 0.9608383178710938
Compression ratio: 11.65131531932126


In [3]:
x_test = np.load('data/mfcc_data/x_test.npy')
y_test = np.load('data/split_data/y_test.npy')

interpreter = tf.lite.Interpreter(model_path=converted_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']

acc = 0
true_labels = []
pred_labels = []
pred_probs = []

for i in tqdm(range(len(x_test))):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])

    true_label = np.argmax(y_test[i])
    pred_label = np.argmax(output_data)
    pred_prob = output_data[0]

    true_labels.append(true_label)
    pred_labels.append(pred_label)
    pred_probs.append(pred_prob)

    if pred_label == true_label:
        acc += 1

acc = acc / len(x_test)
print(f'Accuracy: {acc * 100:.2f}%')

true_labels = np.array(true_labels)
pred_probs = np.array(pred_probs)

auc = roc_auc_score(true_labels, pred_probs, multi_class='ovr')
print(f'AUC: {auc:.4f}')

recall = recall_score(true_labels, pred_labels, average='macro')
print(f'Recall: {recall:.4f}')


100%|██████████| 10583/10583 [00:13<00:00, 770.37it/s]


Accuracy: 76.43%
AUC: 0.9872
Recall: 0.7442
